### 절대 모멘텀
- 파생변수 'STD-YM' 생성 -> index에서 년-월 추출하여 대입
- 'STD-YM' 별 마지막날의 데이터를 month_last_df 저장
- 전월의 수정주가 값을 가진 파생변수를 생성 
- 전년도의 수정주가 값을 가진 파생변수를 생성
- 전월의 데이터와 전년도의 데이터를 기준으로 하여 거래 내역 생성
- 수익율 계산

In [18]:
import pandas as pd 
import numpy as np 
from datetime import datetime

In [3]:
df = pd.read_csv("../../csv/MSFT.csv", index_col='Date')

In [4]:
df.head(1)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1986-03-13,0.088542,0.101563,0.088542,0.097222,0.069996,1031788800


In [5]:
# index를 시계열로 변경 
df.index = pd.to_datetime(df.index, format='%Y-%m-%d')

In [6]:
# index의 데이터에서 년-월 추출하여 새로운 파생변수 STD-YM에 대입
df['STD-YM'] = df.index.strftime('%Y-%m')

In [7]:
df.iloc[10:20]

,Open,High,Low,Close,Adj Close,Volume,STD-YM
Date,,,,,,,
1986-03-27,0.094618,0.096354,0.094618,0.096354,0.069371,16848000,1986-03
1986-03-31,0.096354,0.096354,0.093750,0.095486,0.068746,12873600,1986-03
1986-04-01,0.095486,0.095486,0.094618,0.094618,0.068121,11088000,1986-04
1986-04-02,0.094618,0.097222,0.094618,0.095486,0.068746,27014400,1986-04
1986-04-03,0.096354,0.098958,0.096354,0.096354,0.069371,23040000,1986-04
1986-04-04,0.096354,0.097222,0.096354,0.096354,0.069371,26582400,1986-04
1986-04-07,0.096354,0.097222,0.092882,0.094618,0.068121,16560000,1986-04
1986-04-08,0.094618,0.097222,0.094618,0.095486,0.068746,10252800,1986-04
1986-04-09,0.095486,0.098090,0.095486,0.097222,0.069996,12153600,1986-04


In [8]:
# 월의 말일의 조건식
# 다음 행의 STD-YM과 오늘의 STD-YM의 값이 다른경우
flag = df['STD-YM'] != df.shift(-1)['STD-YM']
df.loc[flag,]

,Open,High,Low,Close,Adj Close,Volume,STD-YM
Date,,,,,,,
1986-03-31,0.096354,0.096354,0.093750,0.095486,0.068746,12873600,1986-03
1986-04-30,0.114583,0.115451,0.109375,0.111979,0.080620,30902400,1986-04
1986-05-30,0.118056,0.123264,0.118056,0.121528,0.087495,27072000,1986-05
1986-06-30,0.103299,0.109375,0.102431,0.106771,0.076870,62352000,1986-06
1986-07-31,0.099826,0.100694,0.098958,0.098958,0.071246,15638400,1986-07
...,...,...,...,...,...,...,...
2019-02-28,112.040001,112.879997,111.730003,112.029999,111.616837,29083900,2019-02
2019-03-29,118.070000,118.320000,116.959999,117.940002,117.505043,25399800,2019-03
2019-04-30,129.809998,130.699997,129.389999,130.600006,130.118362,24166500,2019-04


In [9]:
# 필터를 이용하여 마지막날의 데이터를 추출 
_list =  df['STD-YM'].unique()

month_last_df = pd.DataFrame()

for i in _list:
    flag = df['STD-YM'] == i
    data = df.loc[flag,].tail(1)
    month_last_df = pd.concat([month_last_df, data], axis=0)

In [10]:
month_last_df

,Open,High,Low,Close,Adj Close,Volume,STD-YM
Date,,,,,,,
1986-03-31,0.096354,0.096354,0.093750,0.095486,0.068746,12873600,1986-03
1986-04-30,0.114583,0.115451,0.109375,0.111979,0.080620,30902400,1986-04
1986-05-30,0.118056,0.123264,0.118056,0.121528,0.087495,27072000,1986-05
1986-06-30,0.103299,0.109375,0.102431,0.106771,0.076870,62352000,1986-06
1986-07-31,0.099826,0.100694,0.098958,0.098958,0.071246,15638400,1986-07
...,...,...,...,...,...,...,...
2019-02-28,112.040001,112.879997,111.730003,112.029999,111.616837,29083900,2019-02
2019-03-29,118.070000,118.320000,116.959999,117.940002,117.505043,25399800,2019-03
2019-04-30,129.809998,130.699997,129.389999,130.600006,130.118362,24166500,2019-04


In [11]:
# 전월의 수정 주가를 BF_1M 파생변수에 대입 
# 전년도의 수정주가를 BF_12M 파생변수에 대입 
# 두개의 파생변수는 결측치는 0으로 대체
month_last_df['BF_1M'] = month_last_df['Adj Close'].shift(1).fillna(0)
month_last_df['BF_12M'] = month_last_df['Adj Close'].shift(12).fillna(0)

- 거래 내역 생성 
    - (전월의 수정주가 / 전년도의 수정주가) - 1 의 값이 0보다 크고 무한대가 아닌 경우

In [12]:
df['trade'] = ""

for i in month_last_df.index:
    signal = ""

    # 절대 모멘텀 계산식 
    momentum_index = month_last_df.loc[i, 'BF_1M'] /\
        month_last_df.loc[i, 'BF_12M'] - 1
    flag = True if ( (momentum_index > 0) & (momentum_index != np.inf) ) \
        else False
    
    if flag:
        signal = "buy"
    
    print(f"날짜 : {i}, 모멘텀 인덱스 : {momentum_index}, flag : {flag}, signal : {signal}")
    df.loc[i:, 'trade'] = signal


날짜 : 1986-03-31 00:00:00, 모멘텀 인덱스 : nan, flag : False, signal : 
날짜 : 1986-04-30 00:00:00, 모멘텀 인덱스 : inf, flag : False, signal : 
날짜 : 1986-05-30 00:00:00, 모멘텀 인덱스 : inf, flag : False, signal : 
날짜 : 1986-06-30 00:00:00, 모멘텀 인덱스 : inf, flag : False, signal : 
날짜 : 1986-07-31 00:00:00, 모멘텀 인덱스 : inf, flag : False, signal : 
날짜 : 1986-08-29 00:00:00, 모멘텀 인덱스 : inf, flag : False, signal : 
날짜 : 1986-09-30 00:00:00, 모멘텀 인덱스 : inf, flag : False, signal : 
날짜 : 1986-10-31 00:00:00, 모멘텀 인덱스 : inf, flag : False, signal : 
날짜 : 1986-11-28 00:00:00, 모멘텀 인덱스 : inf, flag : False, signal : 
날짜 : 1986-12-31 00:00:00, 모멘텀 인덱스 : inf, flag : False, signal : 
날짜 : 1987-01-30 00:00:00, 모멘텀 인덱스 : inf, flag : False, signal : 
날짜 : 1987-02-27 00:00:00, 모멘텀 인덱스 : inf, flag : False, signal : 
날짜 : 1987-03-31 00:00:00, 모멘텀 인덱스 : 1.7909114712128704, flag : True, signal : buy
날짜 : 1987-04-30 00:00:00, 모멘텀 인덱스 : 2.0000124038700076, flag : True, signal : buy
날짜 : 1987-05-29 00:00:00, 모멘텀 인덱스 : 1.971415509457683, f

C:\Users\moons\AppData\Local\Temp\ipykernel_1924\238936273.py:7: RuntimeWarning: invalid value encountered in scalar divide
  momentum_index = month_last_df.loc[i, 'BF_1M'] /\
C:\Users\moons\AppData\Local\Temp\ipykernel_1924\238936273.py:7: RuntimeWarning: divide by zero encountered in scalar divide
  momentum_index = month_last_df.loc[i, 'BF_1M'] /\


In [13]:
df['trade'].value_counts()

trade
buy    6005
       2384
Name: count, dtype: int64

In [14]:
df['rtn'] = 1
for i in df.index:
    # 매수 조건식 (전날의 trade가 "" 오늘의 trade가 "buy")
    if (df.shift().loc[i, 'trade'] == "") & (df.loc[i, 'trade'] == "buy"):
        buy = df.loc[i, 'Adj Close']
        print(f"매수일 : {i}, 매수가 : {buy}")
    # 매도 조건식 (전날의 trade가 "buy" 오늘의 trade가 "")
    elif (df.shift().loc[i, 'trade'] == "buy") & (df.loc[i, 'trade'] == ""):
        sell = df.loc[i, 'Adj Close']
        rtn = sell / buy
        df.loc[i, 'rtn'] = rtn
        print(f"매도일 : {i}, 매도가 : {sell}, 수익율 : {rtn}")

매수일 : 1987-03-31 00:00:00, 매수가 : 0.241861
매도일 : 1988-05-31 00:00:00, 매도가 : 0.289983, 수익율 : 1.1989655215185582


C:\Users\moons\AppData\Local\Temp\ipykernel_1924\2680049231.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1.1989655215185582' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'rtn'] = rtn


매수일 : 1988-06-30 00:00:00, 매수가 : 0.33498
매도일 : 1988-09-30 00:00:00, 매도가 : 0.261234, 수익율 : 0.7798495432563138
매수일 : 1988-10-31 00:00:00, 매수가 : 0.244985
매도일 : 1988-12-30 00:00:00, 매도가 : 0.266234, 수익율 : 1.0867359226075066
매수일 : 1989-02-28 00:00:00, 매수가 : 0.297482
매도일 : 1989-04-28 00:00:00, 매도가 : 0.279358, 수익율 : 0.939075305396629
매수일 : 1989-08-31 00:00:00, 매수가 : 0.293733
매도일 : 1993-08-31 00:00:00, 매도가 : 1.690211, 수익율 : 5.754242798732181
매수일 : 1994-02-28 00:00:00, 매수가 : 1.85614
매도일 : 1994-03-31 00:00:00, 매도가 : 1.906761, 수익율 : 1.0272721885202625
매수일 : 1994-06-30 00:00:00, 매수가 : 2.322986
매도일 : 2000-03-31 00:00:00, 매도가 : 38.247726, 수익율 : 16.464897334723496
매수일 : 2000-04-28 00:00:00, 매수가 : 25.108511
매도일 : 2000-05-31 00:00:00, 매도가 : 22.521158, 수익율 : 0.8969531486753635
매수일 : 2001-05-31 00:00:00, 매수가 : 24.903316
매도일 : 2001-06-29 00:00:00, 매도가 : 26.278425, 수익율 : 1.055217907526853
매수일 : 2001-07-31 00:00:00, 매수가 : 23.826982
매도일 : 2001-08-31 00:00:00, 매도가 : 20.536774, 수익율 : 0.8619125158192507
매수일 : 20

In [15]:
df['acc_rtn'] = df['rtn'].cumprod()

In [16]:
df.tail()

,Open,High,Low,Close,Adj Close,Volume,STD-YM,trade,rtn,acc_rtn
Date,,,,,,,,,,
2019-06-18,134.190002,135.240005,133.570007,135.160004,135.160004,25934500,2019-06,buy,1.0,72.288746
2019-06-19,135.000000,135.929993,133.809998,135.690002,135.690002,23744400,2019-06,buy,1.0,72.288746
2019-06-20,137.449997,137.660004,135.720001,136.949997,136.949997,33042600,2019-06,buy,1.0,72.288746
2019-06-21,136.580002,137.729996,136.460007,136.970001,136.970001,36348400,2019-06,buy,1.0,72.288746
2019-06-24,137.000000,138.389999,137.020004,138.289993,138.289993,9131553,2019-06,buy,1.0,72.288746


### 절대 모멘텀 함수화
1. STD-YM 생성하는 함수 
    - 매개변수 2개 (데이터프레임, 기준이 되는 컬럼)
    - 데이터프레임의 복사본을 생성
    - 컬럼에 Date가 포함되어있는지 확인하고 컬럼에 Date 존재한다면 Date를 인덱스로 변환
    - 인덱스를 시계열 데이터로 변경 
    - 데이터중 결측치나 무한대 값을 제외 
    - 기준이 되는 컬럼을 제외한 나머지 컬럼은 모두 삭제
    - 'STD-YM' 컬럼을 생성하여 index에서 '년-월'데이터를 추출하여 대입 
    - 수정이 된 데이터프레임을 리턴

In [17]:
def create_YM(
        _df, 
        _col = 'Adj Close'
):
    df = _df.copy()
    # Date가 컬럼에 포함되어있는가?
    if 'Date' in df.columns:
        # 포함되어있다면 Date를 인덱스로 변환 
        df.set_index('Date', inplace=True)
    # 인덱스를 시계열데이터로 변경
    df.index = pd.to_datetime(df.index, format='%Y-%m-%d')
    # 결측치, 무한대 데이터를 제거 기준이되는 컬럼만 두고 나머지 모두 제거 
    flag = df.isin([np.nan, np.inf, -np.inf]).any(axis=1)
    df = df.loc[~flag, [_col]]
    # 파생변수 STD-YM 생성
    df['STD-YM'] = df.index.strftime('%Y-%m')

    return df


In [19]:
df = pd.read_csv('../../csv/AAPL.csv')

In [21]:
ym_df = create_YM(df)

- 두번째 함수 생성
    - 매개변수 5개 (데이터프레임, 시작시간, 종료시간, 모멘텀 기간, 기준시점)
    - 시작시간의 기본값은 2010-01-01
    - 종료시간의 기본값은 오늘 날짜
    - 모멘텀 기간의 기본값은 12
    - 기준시점의 기본값은 1이고 0으로 변경시 월초의 데이터로 구성 
    - 기준시점의 값에 따라 (월말|월초) 데이터만 모아서 새로운 데이터프레임 생성
    - 생성된 데이터프레임을 기준으로 BF1 컬럼을 생성하여 전월의 데이터를 대입
    - BF2 컬럼을 생성하여 모멘텀 기간(6 -> 6개월 전) 전의 데이터를 대입
    - 결측치는 0으로 대체
    - 데이터프레임을 시작시간과 종료시간으로 데이터 필터링 
    - 결과를 리턴

In [ ]:
def create_month(
        _df, 
        _start = "2010-01-01", 
        _end = datetime.now(), 
        _momentum = 12, 
        _select = 1
):
    if _select == 1:
        # 월말의 데이터들을 새로운 데이터프레임으로 생성 
        # 현재 행의 년-월과 다음 행의 년-월이 다른 경우 
        flag = _df['STD-YM'] != _df.shift(-1)['STD-YM']
        # df = _df.loc[flag,]
    elif _select == 0:
        flag = _df['STD-YM'] != _df.shift()['STD-YM']
        # df = _df.loc[flag,]
    else :
        return "_select 인자는 0과 1이 가능하다"
    df = _df.loc[flag,]